# 실습 퀴즈: MoE 레이어 커스텀 구현

## 사용 방법
- 각 문제 셀을 읽고, **직접 답을 예측한 후** 풀이 셀을 실행하세요
- 코드 실행 전에 종이에 계산해보는 것을 권장합니다

## 목차
- [Q1: Top-2 Router Softmax 게이팅](#q1)
- [Q2: Expert Dispatch와 Combine](#q2)
- [Q3: Auxiliary Loss 계산](#q3)
- [Q4: Shared Expert 추가](#q4)
- [종합 도전: DeepSeekMoE 스타일 완전한 MoE 레이어](#bonus)

---

In [ ]:
# 퀴즈 환경 설정
import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)
print(f"TensorFlow 버전: {tf.__version__}")
print("퀴즈 환경 준비 완료!")

---
## Q1: Top-2 Router Softmax 게이팅 <a name='q1'></a>

### 문제

4명의 Expert에 대한 라우터 로짓이 $h = [1.0, 3.0, 0.5, 2.0]$입니다.
Top-2를 선택하고 재정규화된 게이팅 가중치를 계산하세요.

**여러분의 예측:** 선택된 Expert 인덱스는 `?`, 게이팅 가중치는 `?` 입니다.

In [ ]:
# ── Q1 풀이 ──────────────────────────────────────────────────
print("=" * 45)
print("Q1 풀이: Top-2 Router Softmax 게이팅")
print("=" * 45)

h = tf.constant([1.0, 3.0, 0.5, 2.0])
top_k = 2

# Top-2 선택
values, indices = tf.math.top_k(h, k=top_k)
print(f"로짓 h: {h.numpy()}")
print(f"Top-{top_k} 인덱스: {indices.numpy()}")
print(f"Top-{top_k} 값: {values.numpy()}")

# 재정규화 (Top-k에 대해서만 softmax)
gates = tf.nn.softmax(values)
print(f"게이팅 가중치: {gates.numpy()}")
print()
print("[해설]")
print(f"  Top-2: Expert {indices[0].numpy()} (h={values[0].numpy():.1f}), "
      f"Expert {indices[1].numpy()} (h={values[1].numpy():.1f})")
print(f"  g_1 = exp(3) / (exp(3) + exp(2)) = {np.exp(3)/(np.exp(3)+np.exp(2)):.4f}")
print(f"  g_3 = exp(2) / (exp(3) + exp(2)) = {np.exp(2)/(np.exp(3)+np.exp(2)):.4f}")

---
## Q2: Expert Dispatch와 Combine <a name='q2'></a>

### 문제

3개의 토큰, 4명의 Expert, Top-2 기준으로:
- 토큰 0 → Expert [1, 3], 가중치 [0.7, 0.3]
- 토큰 1 → Expert [0, 2], 가중치 [0.6, 0.4]
- 토큰 2 → Expert [1, 0], 가중치 [0.5, 0.5]

Expert별 할당된 토큰 수와 가중합 출력을 계산하세요.

**여러분의 예측:** Expert 1에 할당된 토큰 수는 `?`개 입니다.

In [ ]:
# ── Q2 풀이 ──────────────────────────────────────────────────
print("=" * 45)
print("Q2 풀이: Expert Dispatch와 Combine")
print("=" * 45)

dispatch = {
    0: {'experts': [1, 3], 'gates': [0.7, 0.3]},
    1: {'experts': [0, 2], 'gates': [0.6, 0.4]},
    2: {'experts': [1, 0], 'gates': [0.5, 0.5]},
}

n_experts = 4
expert_loads = {i: [] for i in range(n_experts)}

for tok, info in dispatch.items():
    for exp, gate in zip(info['experts'], info['gates']):
        expert_loads[exp].append((tok, gate))

print("Expert별 할당:")
for exp, assignments in expert_loads.items():
    tokens = [f"토큰{t}(g={g})" for t, g in assignments]
    print(f"  Expert {exp}: {', '.join(tokens) if tokens else '없음'} → {len(assignments)}개 토큰")
print()
print("출력 계산 (가상 Expert 출력 사용):")
d = 4
expert_outputs = {i: np.random.randn(d) for i in range(n_experts)}
for tok, info in dispatch.items():
    combined = np.zeros(d)
    for exp, gate in zip(info['experts'], info['gates']):
        combined += gate * expert_outputs[exp]
    print(f"  토큰 {tok}: {combined[:2]}... (처음 2차원)")
print()
print("[해설]")
print("  Expert 1: 토큰 0, 2 → 2개 (가장 많음)")
print("  Expert 3: 토큰 0만 → 1개")

---
## Q3: Auxiliary Loss 계산 <a name='q3'></a>

### 문제

$N=4$, $\alpha=0.01$, 전체 토큰 16개:
- Expert별 토큰 수: [8, 4, 2, 2]
- 평균 라우팅 확률 $P = [0.4, 0.3, 0.15, 0.15]$

$L_{aux} = \alpha \cdot N \sum_{i=1}^{N} f_i \cdot P_i$를 계산하세요.

**여러분의 예측:** $L_{aux}$는 `?` 입니다.

In [ ]:
# ── Q3 풀이 ──────────────────────────────────────────────────
print("=" * 45)
print("Q3 풀이: Auxiliary Loss 계산")
print("=" * 45)

N = 4
alpha = 0.01
counts = np.array([8, 4, 2, 2], dtype=float)
T = counts.sum()
f = counts / T  # 실제 분배 비율
P = np.array([0.4, 0.3, 0.15, 0.15])  # 평균 라우팅 확률

L_aux = alpha * N * np.sum(f * P)

print(f"N = {N}, alpha = {alpha}")
print(f"토큰 수: {counts.astype(int)}, 총 T = {int(T)}")
print(f"f (실제 비율): {f}")
print(f"P (라우팅 확률): {P}")
print()
print(f"f · P = {f * P}")
print(f"sum(f · P) = {np.sum(f * P):.4f}")
print(f"L_aux = {alpha} × {N} × {np.sum(f * P):.4f} = {L_aux:.6f}")
print()

# 균등 분배 시 비교
L_uniform = alpha * N * N * (1/N)**2
print(f"균등 분배 시 L_aux = {L_uniform:.6f}")
print(f"현재 불균형도: L_aux / L_uniform = {L_aux/L_uniform:.2f}x")
print()
print("[해설]")
print(f"  현재 L_aux ({L_aux:.6f}) > 균등 L_aux ({L_uniform:.6f})")
print(f"  → 불균형 상태. Loss를 최소화하면 자동으로 균형 유도!")

---
## Q4: Shared Expert 추가 <a name='q4'></a>

### 문제

기존 MoE에 Shared Expert 1개를 추가합니다. 입력 $x$에 대해:
- Shared Expert 출력: $E_s(x)$
- Routed Expert 출력: $\sum g_i E_i^r(x)$ (Top-2)

최종 출력 공식과, Shared Expert 추가가 파라미터 수에 미치는 영향을 분석하세요.

**여러분의 예측:** Shared Expert 추가 시 활성 파라미터 증가량은 `?`% 입니다.

In [ ]:
# ── Q4 풀이 ──────────────────────────────────────────────────
print("=" * 45)
print("Q4 풀이: Shared Expert 추가 분석")
print("=" * 45)

d_model = 256
d_ff = 512
n_routed = 8
top_k = 2

# Expert당 파라미터 (FFN: up + down)
expert_params = d_model * d_ff + d_ff * d_model
shared_params = expert_params  # Shared도 같은 구조

# 활성 파라미터 (처리 시)
active_without_shared = top_k * expert_params
active_with_shared = (1 + top_k) * expert_params

print(f"설정: d_model={d_model}, d_ff={d_ff}, N_routed={n_routed}, Top-{top_k}")
print(f"Expert당 파라미터: {expert_params:,}")
print()
print(f"{'구성':<25} | {'총 파라미터':>12} | {'활성 파라미터':>12}")
print("-" * 55)
print(f"{'MoE (Shared 없음)':<25} | {n_routed*expert_params:>12,} | {active_without_shared:>12,}")
print(f"{'DeepSeekMoE (Shared 1)':<25} | {(n_routed+1)*expert_params:>12,} | {active_with_shared:>12,}")
print()
increase = (active_with_shared / active_without_shared - 1) * 100
print(f"Shared Expert 추가 시 활성 파라미터 증가: {increase:.0f}%")
print(f"  → Shared Expert가 '공통 지식'을 담당하므로 Top-k 감소 가능")
print()
print("최종 출력:")
print("  y = E_s(x) + sum(g_i * E_i^r(x)) for i in Top-k")
print("  Shared는 항상 활성 → 공통 패턴 학습")
print("  Routed는 선택적 → 전문 패턴 학습")

---
## 종합 도전: DeepSeekMoE 스타일 완전한 MoE 레이어 <a name='bonus'></a>

### 문제

다음 조건으로 **완전한 DeepSeekMoE 레이어**를 구현하세요:
1. Shared Expert 1개 + Routed Expert 4개
2. Top-2 라우팅 (Softmax 게이팅)
3. Auxiliary-Loss-Free 편향 보정
4. 배치 처리 지원

In [ ]:
# ── 종합 도전 풀이: DeepSeekMoE 레이어 ────────────────────────
print("=" * 55)
print("종합 도전: DeepSeekMoE 스타일 MoE 레이어")
print("=" * 55)

class MiniDeepSeekMoE(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff, n_routed=4, top_k=2):
        super().__init__()
        self.n_routed = n_routed
        self.top_k = top_k
        
        # Shared Expert
        self.shared = tf.keras.Sequential([
            tf.keras.layers.Dense(d_ff, activation='relu'),
            tf.keras.layers.Dense(d_model)
        ])
        
        # Routed Experts
        self.experts = [
            tf.keras.Sequential([
                tf.keras.layers.Dense(d_ff, activation='relu'),
                tf.keras.layers.Dense(d_model)
            ]) for _ in range(n_routed)
        ]
        
        # Router
        self.router = tf.keras.layers.Dense(n_routed, use_bias=False)
        
        # Aux-Free bias
        self.bias = tf.Variable(tf.zeros(n_routed), trainable=False)
    
    def call(self, x):
        # Shared output
        out = self.shared(x)
        
        # Routing
        logits = self.router(x) + self.bias
        top_vals, top_idx = tf.math.top_k(logits, k=self.top_k)
        gates = tf.nn.softmax(top_vals, axis=-1)
        
        # Routed output
        for k in range(self.top_k):
            idx = top_idx[:, :, k]
            g = gates[:, :, k:k+1]
            for e in range(self.n_routed):
                mask = tf.cast(tf.equal(idx, e), tf.float32)[:, :, tf.newaxis]
                if tf.reduce_sum(mask) > 0:
                    out += self.experts[e](x) * mask * g
        
        return out

# 테스트
d_model, d_ff = 64, 128
moe = MiniDeepSeekMoE(d_model, d_ff, n_routed=4, top_k=2)
x = tf.random.normal((2, 8, d_model))
y = moe(x)

total = sum(tf.size(v).numpy() for v in moe.trainable_variables)
print(f"입력: {x.shape}")
print(f"출력: {y.shape}")
print(f"총 파라미터: {total:,}")
print(f"활성 Expert: 1(shared) + 2(routed) = 3")
print()
print("🎉 DeepSeekMoE 스타일 레이어 구현 완료!")
print("   Shared Expert가 공통 지식을, Routed Expert가 전문 지식을 담당합니다.")